In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
data_dir = '\dataset'
labels_dir = data_dir+'\\train.csv'

Y = pd.read_csv(os.getcwd()+labels_dir,dtype=str)

In [3]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import shutil

In [4]:
labels = Y['Class']
unique_labels = labels.unique()
image_names = Y['Image']

In [5]:
## Creating Directories
def create_directories(unique_labels,directory):
    
    try:
        for unique in unique_labels:
            os.mkdir(directory+unique)
    
    except:
        print("Some Error Occured")
        
    else:
        print("Created Corresponding Directories")

#create_directories(unique_labels,'.'+data_dir+'\\train\\')

In [6]:
image_dir  = '.'+data_dir+'\\train\\' 

## Segregate Images to Directories
def move_images(image_names):
    for i in range(len(image_names)):
        try:        
            shutil.move(image_dir+image_names[i],image_dir+labels[i])
            print("%s Copied to %s"%(image_names[i],labels[i]))
        except:
            print("Error in Copying")
            continue

#move_images(image_names)

In [7]:
from tensorflow import keras

def load_model():
    return keras.models.load_model('Xception_update')

def save_model(model):
    model.save("Xception_update")

In [8]:
#model = load_model()

## Image Generators for Training and Validation

In [9]:
datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    image_dir,
    target_size=(100, 100),
    batch_size=16,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = datagen.flow_from_directory(
    image_dir, # same directory as training data
    target_size=(100, 100),
    batch_size=16,
    class_mode='categorical',
    subset='validation')

Found 5177 images belonging to 6 classes.
Found 1292 images belonging to 6 classes.


In [10]:
import tensorflow as tf

In [20]:
def generate_model():

    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(128,activation='relu'))
    model.add(tf.keras.layers.Dense(36,activation='relu'))
    model.add(tf.keras.layers.Dense(6,activation='softmax'))

    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
    
    return model

model = generate_model()

In [21]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=1)

In [22]:
model.fit(train_generator,validation_data=validation_generator,callbacks=[early_stop],epochs=5)

Epoch 1/5
322/324 [============================>.] - ETA: 2s - loss: 1.1158 - acc: 0.6375

KeyboardInterrupt: 

In [13]:
## Saving The Model
save_model(model)

NameError: name 'model' is not defined

## Generating Predictions

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    './dataset/test_1',
    target_size=(100, 100),
    batch_size=1)

In [ ]:
preds=model.predict(test_generator)
predicted_class_indices=np.argmax(preds,axis=1)

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
filenames=test_generator.filenames
for i in range(len(filenames)):filenames[i] = filenames[i][5:]

results=pd.DataFrame({"Image":filenames,
                      "Class":predictions})

In [ ]:
results